用于连接ai程序应用\
可以远程托管\
MCP (Model Context Protocol)\
\
MCP 是一个基于 JSON-RPC 2.0 的协议，支持客户端和服务器之间的通信，主要用于模型（例如 AI 模型）和工具的交互。MCP 负责 数据层（通过 JSON-RPC 进行消息交换）和 传输层（支持 HTTP 或 stdin/stdout 的通信）之间的通信。\
\
MCP 服务器通过暴露工具、资源和提示等原语，允许客户端与模型和资源进行交互。\



#### 特点
- 多源上下文， 且具有一致性
- 通过上下文，动态调用
- 状态性
- 双向通信

#### 功能
- 1m+ 令牌，长文档和对话
- 预制在文档工作能力
- 批量处理请求， 节省计算成本
- 提供原文档信息（citation）
- 动态管理上下文，更新指令管理token数量（clearing tool）
- 加强reasoning 能力， 多步思考
- ** 文档api： pdf（多模态，文字 + 图像）, docs, images 不用重复上传**
- 一小时对话缓存

使用工作工具 
跨越文档数据库和organization
读取日历和上面的标注
使用figma做设计
设计3d建模 + 打印

#### 用于：
建立服务器/建立连接器
- 调用ai工具
- 让ai app 和 agent使用数据源，工具 提高性能
- 可以直接改动文本数据库 （access -> action）

建立mcp server：提供外部资源能力上下文 开发部署资源工具提供端
- 创建， 暴露功能
- json-rpc 格式
- 定义能做什么\

建立mcp client：消费这些能力，如claude 本身
- 连接调度转发
- claude 接口管理
- 连接一个或多个mcp server
- 管理认证，会话，上下文同步

In [1]:
## build server


#### mcp server
- data layer: json-rpc 格式，
     - 生命周期管理：客户端服务器连接初始化，能力协商和连接终止
     - 服务器功能：定义核心功能，ai 可以使用的工具
     - 客户端功能： 诉诸语言模型采样，向用户征求（llm的）输入以及客户端信息
     - 使用功能：实时更新提醒，长时间运行的进度追踪

- transport layer: 设计交流模式
    - 客户和服务器间的交流模式和认证
    - mcp服务器之间的连接的建立，信息结构，安全连接 
    - 支持两种传输方式
        - stdio stdin/stdout 提供最佳性能且没有网络开销
        - streambale http POST/GET 并可选支持服务器推送事件（Server-Sent Events）进行流式传输。该传输机制支持远程服务器通信，并支持标准的HTTP身份验证方法，包括持有者令牌（Bearer Tokens）、API密钥和自定义头部。MCP建议使用OAuth获取身份验证令牌。

- 数据层协议 data layer protocal
mcp 的核心部分是定义客户端和服务器之间的架构和语义（schema and sematics），定义了开发者可以共享从客户端到服务器的上下文信息
    - 生命周期管理：
        有状态特协议需要生命周期管理，协商客户端和服务器支持的能力，包括连接初始化、能力协商、连接终止等
    - **原语 primitives**：
        定义了客户端和服务器可以提供给对方的内容，制定了可与ai共享的上下文信息信息以及可以执行的操作范围
        - *服务器暴露*   - 工具 tool： 可执行函数，执行操作例如文件操作，api调用，数据库查询
                     - 资源 resource：上下文信息数据库如文件内容，api响应，数据库记录
                     - 提示 prompt： 可回收再利用的语言交互模版（system prompts 系统提示，few-shot prompts 少样本- 提示）
        每种原语类型都有与之相关的发现（/list）、检索（/get）和执行（tools/call）方法。MCP客户端将使用*/list方法来发现可用的原语。例如，客户端可以先列出所有可用的工具（tools/list），然后执行它们。这种设计允许动态列出原语。
        - *客户端暴露* - 采样 sampling：让服务器可以从客户端的ai app生成语言模型， 服务器作作者想获取llm恢复但依然想工具                        独立于llm，不需要包含llm的sdk。 可以用sampling/complete方法获得客户端ai app生成
                    - 引导 elicitation：允许服务器请求用户获得额外信息或确认某个操作。使用 elicitation/request 方法
                    - 日志记录 logging：日志发送到客户端，用于调试和监控
    - 通知 notifications：
        协议支持实时通知启用服务器和客户端之间状态的更新， 例如服务器的可用工具发生变化时或现有工具被修改， 服务器可以发送工具更新通知，告知一连接的客户端。


#### MCP CLIENT





### 核心功能：
- sampling：让服务器从llm 获取信息
- roots：client端的上下文范围
- elicitation：向用户取得信息和确认

这一部分相对简单，主要在信息处理和根目录的设计。\

根目录是重要的功能实现，他控制上下文范围，事故预防，组织工作流。

### 举例：旅行计划MCP
一个旅行计划/旅行社需要有多个行程确定的agent共同完成。要设计一个工作环境/上下文可以计划旅行的不同方面。\
这个文件系统可能包括以下的范围（roots）：\
- ‘file:///users/agent/travel-planning’ 主要上下文，包括所有旅行文件
- ‘file:///users/agent/travel-templates’可再利用的行程模版和资源
- ‘file:///users/agent/client-documents’用户旅行证件信息
根目录不作为保安，而是作为联通server和client的桥梁，提供上下文管理和资源共享。

#### elicitation 信息引出

